### Black Box Attacks and Defenses 

### Experiment 4 
> * Purpose: Testing defenses 

In [1]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.models as models 

import numpy as np

from art.estimators.classification import PyTorchClassifier

from PIL import Image
from torchvision import transforms

In [2]:
import ast 
import pandas as pd 

In [3]:
import matplotlib.pyplot as plt 

from datetime import datetime ## important 
import time 
import warnings
warnings.filterwarnings('ignore')

In [5]:
import torch
# torch.cuda.set_device(0)
# torch.cuda.current_device()

#### Attacks from ART 

In [6]:
from art.attacks.evasion import ProjectedGradientDescent
from art.attacks.evasion import FastGradientMethod
from art.attacks.evasion import NewtonFool 
from art.attacks.evasion import CarliniL2Method
from art.attacks.evasion import SimBA
from art.attacks.evasion import BoundaryAttack 
from art.attacks.evasion import HopSkipJump

In [7]:
from skimage.metrics import structural_similarity as ssim

#### Some methods and variables 

In [8]:
preprocess = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [9]:
def softmax_activation(inputs): 
    inputs = inputs.tolist()
    exp_values = np.exp(inputs - np.max(inputs)) 
    
    # Normalize 
    probabilities = exp_values / np.sum(exp_values)
    return probabilities

In [10]:
def compare_images(imageA, imageB):
    return 1 - ssim(imageA, imageB, multichannel=True)

In [11]:
def calc_L_dist(adv_map, targ_map):
    n = len(adv_map)
    l1_dist = np.linalg.norm(adv_map.reshape((n, -1)) - targ_map.reshape((n, -1)), 1, axis=1) 
    
    l1_dist = l1_dist[0]/(224*224)
    return l1_dist

#### Defense Methods 

In [12]:
class FloatToIntSqueezing(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x, max_int, vmin, vmax):
        # here assuming 0 =< x =< 1
        x = (x - vmin) / (vmax - vmin)
        x = torch.round(x * max_int) / max_int
        return x * (vmax - vmin) + vmin

    @staticmethod
    def backward(ctx, grad_output):
        raise NotImplementedError(
            "backward not implemented", FloatToIntSqueezing)

In [13]:
import torch.nn as nn


class Processor(nn.Module):
    """
    Processor
    """
    def __init__(self):
        super(Processor, self).__init__()

    def forward(self, x):
        return x

    def extra_repr(self):
        return 'EmptyDefense (Identity)' 

In [14]:
class BitSqueezing(Processor):
    """
    Bit Squeezing.
    :param bit_depth: bit depth.
    :param vmin: min value.
    :param vmax: max value.
    """

    def __init__(self, bit_depth, vmin=0., vmax=1.):
        super(BitSqueezing, self).__init__()

        self.bit_depth = bit_depth
        self.max_int = 2 ** self.bit_depth - 1
        self.vmin = vmin
        self.vmax = vmax

    def forward(self, x):
        return FloatToIntSqueezing.apply(
            x, self.max_int, self.vmin, self.vmax)


class BinaryFilter(BitSqueezing):
    """
    Binary Filter.
    :param vmin: min value.
    :param vmax: max value.
    """

    def __init__(self, vmin=0., vmax=1.):
        super(BinaryFilter, self).__init__(bit_depth=1, vmin=vmin, vmax=vmax)

In [15]:
from torch.nn.modules.utils import _quadruple

def _is_even(x):
    return int(x) % 2 == 0

In [16]:
class MedianSmoothing2D(Processor):
    """
    Median Smoothing 2D.
    :param kernel_size: aperture linear size; must be odd and greater than 1.
    :param stride: stride of the convolution.
    """

    def __init__(self, kernel_size=3, stride=1):
        super(MedianSmoothing2D, self).__init__()
        self.kernel_size = kernel_size
        self.stride = stride
        padding = int(kernel_size) // 2
        if _is_even(kernel_size):
            # both ways of padding should be fine here
            # self.padding = (padding, 0, padding, 0)
            self.padding = (0, padding, 0, padding)
        else:
            self.padding = _quadruple(padding)

    def forward(self, x):
        x = F.pad(x, pad=self.padding, mode="reflect")
        x = x.unfold(2, self.kernel_size, self.stride)
        x = x.unfold(3, self.kernel_size, self.stride)
        x = x.contiguous().view(x.shape[:4] + (-1,)).median(dim=-1)[0]
        return x
  


#### Uploading files 

In [17]:
### Variables 
DATA_DIR = "data/imagenetv2/imagenetv2-top-images-format-val"
# selected_images_file_path = 'data/1000_images_paths_3_filter65.txt'
selected_images_file_path = 'data/200_images_paths_filter65.txt' 

#### Loading images selected image paths 

In [18]:
selected_images_list_file = open(selected_images_file_path, "r")
loaded_image_paths_list = selected_images_list_file.read()

### Converst file content into list using ast
loaded_image_paths_list = ast.literal_eval(loaded_image_paths_list)

In [19]:
len(loaded_image_paths_list)

200

### Main method

In [20]:
def experiment1_method(target_classifier, attack, defense, image_list): 
    column_list = ['image', 'benign_pred', 'benign_conf', 'adv_pred', 'adv_conf', 'attack_time', 'noise_ratio'] 
    results_df =  pd.DataFrame(columns = column_list) 
    
    for image in image_list: 
        input_image = Image.open(DATA_DIR + image)
        input_tensor = preprocess(input_image)
        input_batch = input_tensor.unsqueeze(0).numpy().astype(np.float32)
        
        preds = target_classifier.predict(input_batch)  
        predicted_label = np.argmax(preds, axis=1)[0] 
        
        confidence = round(np.max(softmax_activation(preds), axis=1)[0], 3) 
        
        ## attack 
        start = time.time() 
        adv_image = attack.generate(x=input_batch)
        attack_time = time.time()-start 
        
        noise_ratio = compare_images(input_batch[0].transpose(1,2,0), adv_image[0].transpose(1,2,0))
        noise_ratio = round(noise_ratio, 5) 
        
        l1_distance = calc_L_dist(adv_image, input_batch)
        
        adv_prediction = target_classifier.predict(adv_image)  
        adv_predicted_label = np.argmax(adv_prediction, axis=1)[0] 
        adv_confidence = round(np.max(softmax_activation(adv_prediction), axis=1)[0], 3)  
        
        ## defense 
        ben_image = torch.from_numpy(input_batch) 
        adv_image = torch.from_numpy(adv_image)
        
        def_ben_image = defense(ben_image) 
        
        start = time.time()  
        def_adv_image = defense(adv_image) 
        defense_time = time.time()-start 
        
        def_ben_noise_rate = compare_images(input_batch[0].transpose(1,2,0), 
                                        def_ben_image[0].numpy().astype(np.float32).transpose(1,2,0))
        def_ben_noise_rate = round(def_ben_noise_rate, 5)
        
        def_noise_rate = compare_images(input_batch[0].transpose(1,2,0), 
                                        def_adv_image[0].numpy().astype(np.float32).transpose(1,2,0))
        def_noise_rate = round(def_noise_rate, 5) 
        
        def_l1_distance = calc_L_dist(def_adv_image, input_batch) 
        
        def_ben_image = def_ben_image.numpy().astype(np.float32) 
        def_ben_preds = target_classifier.predict(def_ben_image)  
        def_ben_predicted_label = np.argmax(def_ben_preds, axis=1)[0]
        def_ben_confidence = round(np.max(softmax_activation(def_ben_preds), axis=1)[0], 3)  
        
        def_adv_image = def_adv_image.numpy().astype(np.float32) 
        def_adv_preds = target_classifier.predict(def_adv_image)  
        def_acv_predicted_label = np.argmax(def_adv_preds, axis=1)[0]
        def_adv_confidence = round(np.max(softmax_activation(def_adv_preds), axis=1)[0], 3)  
        
        
        results_df = results_df.append({'image': image, 
                                       'benign_pred': predicted_label, 
                                       'benign_conf': confidence, 
                                       'def_ben_noise_rate': def_ben_noise_rate, 
                                       'adv_pred': adv_predicted_label,
                                       'adv_conf': adv_confidence, 
                                       'attack_time': attack_time, 
                                       'noise_ratio': noise_ratio, 
                                       'l1_distance': l1_distance, 
                                       'def_ben_pred': def_ben_predicted_label, 
                                       'def_ben_conf': def_ben_confidence, 
                                       'def_adv_pred': def_acv_predicted_label,  
                                       'def_adv_conf': def_adv_confidence, 
                                       'defense_time': defense_time, 
                                       'def_noise_rate': def_noise_rate, 
                                       'def_l1_distance': def_l1_distance,
                                       }, ignore_index=True)
    return results_df

In [21]:
def summarize_result(results, total_images, resultsDF, model_name):
    avg_benign_conf = round(results['benign_conf'].mean(), 3)
    benign_conf_std = round(results['benign_conf'].std(), 3)
    
    attack_success_count = results[results.benign_pred != results.adv_pred]["image"].count()
    attack_failure_count = total_images - attack_success_count 
    
    avg_adv_success_conf = round(results[results.benign_pred != results.adv_pred]["adv_conf"].mean(), 3) 
    adv_success_conf_std = round(results[results.benign_pred != results.adv_pred]["adv_conf"].std(), 3) 
    
    avg_adv_fail_conf = round(results[results.benign_pred == results.adv_pred]["adv_conf"].mean(), 3) 
    adv_fail_conf_std = round(results[results.benign_pred == results.adv_pred]["adv_conf"].std(), 3) 
    
    avg_attack_time = round(results['attack_time'].mean(), 4) 
    attack_time_std = round(results['attack_time'].std(), 4) 
    
    avg_noise_ratio = round(results['noise_ratio'].mean(), 5) 
    noise_ratio_std = round(results['noise_ratio'].std(), 5)
    
    avg_l1_distance = round(results['l1_distance'].mean(), 5)      
    l1_distance_std = round(results['l1_distance'].std(), 5)  
    
    ## defense 
    def_ben_success_count = results[results.benign_pred == results.def_ben_pred]["image"].count()     
    def_ben_failure_count = total_images - def_ben_success_count  
    
    avg_def_ben_success_conf = round(results[results.benign_pred == results.def_ben_pred]["def_ben_conf"].mean(), 3) 
    def_ben_success_conf_std = round(results[results.benign_pred == results.def_ben_pred]["def_ben_conf"].std(), 3) 
    
    def_adv_success_count = results[results.benign_pred == results.def_adv_pred]["image"].count()   
    avg_def_adv_success_conf = round(results[results.benign_pred == results.def_adv_pred]["def_adv_conf"].mean(), 3) 
    def_adv_success_conf_std = round(results[results.benign_pred == results.def_adv_pred]["def_adv_conf"].std(), 3) 
    
    ### 
    avg_def_ben_noise_rate = round(results['def_ben_noise_rate'].mean(), 4) 
    std_def_ben_noise_rate = round(results['def_ben_noise_rate'].std(), 4) 
    
    ### 
    def_succ_adv_success_count = results[(results.benign_pred == results.def_adv_pred) & 
                                    (results.benign_pred != results.adv_pred)]["image"].count()   
    avg_def_succ_adv_success_conf = round(results[(results.benign_pred == results.def_adv_pred) & 
                                                 (results.benign_pred != results.adv_pred)]["def_adv_conf"].mean(), 3) 
    def_succ_adv_success_conf_std = round(results[(results.benign_pred == results.def_adv_pred) & 
                                                 (results.benign_pred != results.adv_pred)]["def_adv_conf"].std(), 3) 
    
    def_fail_adv_success_count = results[(results.benign_pred == results.def_adv_pred) & 
                                    (results.benign_pred == results.adv_pred)]["image"].count()   
    avg_def_fail_adv_success_conf = round(results[(results.benign_pred == results.def_adv_pred) & 
                                                 (results.benign_pred == results.adv_pred)]["def_adv_conf"].mean(), 3) 
    def_fail_adv_success_conf_std = round(results[(results.benign_pred == results.def_adv_pred) & 
                                                 (results.benign_pred == results.adv_pred)]["def_adv_conf"].std(), 3)
    
    avg_defense_time = round(results['defense_time'].mean(), 4) 
    std_defense_time = round(results['defense_time'].std(), 4) 
    
    avg_noise_rate_success_attacks = round(results[results.benign_pred != results.adv_pred]["noise_ratio"].mean(), 3) 
    std_noise_rate_success_attacks = round(results[results.benign_pred != results.adv_pred]["noise_ratio"].std(), 3)
    
    avg_noise_rate_success_attacks_def_s = round(results[(results.benign_pred == results.def_adv_pred) & 
                                                 (results.benign_pred != results.adv_pred)]["def_noise_rate"].mean(), 3) 
    
    std_noise_rate_success_attacks_def_s = round(results[(results.benign_pred == results.def_adv_pred) & 
                                                 (results.benign_pred != results.adv_pred)]["def_noise_rate"].std(), 3)
    
    avg_noise_rate_success_attacks_def_f = round(results[(results.benign_pred != results.def_adv_pred) & 
                                                 (results.benign_pred != results.adv_pred)]["def_noise_rate"].mean(), 3) 
    
    std_noise_rate_success_attacks_def_f = round(results[(results.benign_pred != results.def_adv_pred) & 
                                                 (results.benign_pred != results.adv_pred)]["def_noise_rate"].std(), 3)
    
    
    avg_noise_rate_failed_attacks = round(results[results.benign_pred == results.adv_pred]["noise_ratio"].mean(), 3) 
    std_noise_rate_failed_attacks = round(results[results.benign_pred == results.adv_pred]["noise_ratio"].std(), 3) 
    
    avg_noise_rate_failed_attacks_def = round(results[(results.benign_pred == results.def_adv_pred) & 
                                                 (results.benign_pred == results.adv_pred)]["def_noise_rate"].mean(), 3) 
    
    std_noise_rate_failed_attacks_def = round(results[(results.benign_pred == results.def_adv_pred) & 
                                                 (results.benign_pred == results.adv_pred)]["def_noise_rate"].std(), 3) 
    
    
    #### L1 distances 
    avg_l1_distance_success_attacks_def_s = round(results[(results.benign_pred == results.def_adv_pred) & 
                                                 (results.benign_pred != results.adv_pred)]["def_l1_distance"].mean(), 3) 
    std_l1_distance_success_attacks_def_s = round(results[(results.benign_pred == results.def_adv_pred) & 
                                                 (results.benign_pred != results.adv_pred)]["def_l1_distance"].std(), 3) 
    
    avg_l1_distance_success_attacks_def_f = round(results[(results.benign_pred != results.def_adv_pred) & 
                                                 (results.benign_pred != results.adv_pred)]["def_l1_distance"].mean(), 3) 
    std_l1_distance_success_attacks_def_f = round(results[(results.benign_pred != results.def_adv_pred) & 
                                                 (results.benign_pred != results.adv_pred)]["def_l1_distance"].std(), 3)
    
    
    avg_l1_distance_failed_attacks_def = round(results[(results.benign_pred == results.def_adv_pred) & 
                                                 (results.benign_pred == results.adv_pred)]["def_l1_distance"].mean(), 3) 
    
    std_l1_distance_failed_attacks_def = round(results[(results.benign_pred == results.def_adv_pred) & 
                                                 (results.benign_pred == results.adv_pred)]["def_l1_distance"].std(), 3) 
    
    
    ### calling 
    benign_conf = ("{} ± {}".format(avg_benign_conf, benign_conf_std))
    benign_defence_success_rate = ("{}/{} ({}±{})".format(def_ben_success_count, total_images, avg_def_ben_success_conf, def_ben_success_conf_std))
    benign_noise = ("{} ± {}".format(avg_def_ben_noise_rate, std_def_ben_noise_rate))
    attack_success_count_text = ("{}/{} ({}±{})".format(attack_success_count, total_images, avg_adv_success_conf, adv_success_conf_std))
    noise_success_attacks = ("{} ± {}".format(avg_noise_rate_success_attacks, std_noise_rate_success_attacks))
    attack_failure_count_text = ("{}/{} ({}±{})".format(attack_failure_count, total_images, avg_adv_fail_conf, adv_fail_conf_std))
    noise_failed_attacks = ("{} ± {}".format(avg_noise_rate_failed_attacks, std_noise_rate_failed_attacks))   
    defence_success_s_attacks = ("{}/{} ({}±{})".format(def_succ_adv_success_count, attack_success_count, 
                                                                  avg_def_succ_adv_success_conf, def_succ_adv_success_conf_std))
    noise_s_defence_success_attacks = ("{} ± {}".format(avg_noise_rate_success_attacks_def_s, std_noise_rate_success_attacks_def_s))
    noise_f_defence_success_attacks = ("{} ± {}".format(avg_noise_rate_success_attacks_def_f, std_noise_rate_success_attacks_def_f))
    
    defence_success_f_attacks = ("{}/{} ({}±{})".format(def_fail_adv_success_count, attack_failure_count, 
                                                                  avg_def_fail_adv_success_conf, def_fail_adv_success_conf_std))
    noise_defence_success_f_attacks = ("{} ± {}".format(avg_noise_rate_failed_attacks_def, std_noise_rate_failed_attacks_def))
    defense_time = ("{} ± {}".format(avg_defense_time, std_defense_time))
    
    ### L1 distances 
    l1_s_defence_success_attacks = ("{} ± {}".format(avg_l1_distance_success_attacks_def_s, std_l1_distance_success_attacks_def_s))
    l1_f_defence_success_attacks = ("{} ± {}".format(avg_l1_distance_success_attacks_def_f, std_l1_distance_success_attacks_def_f))
    l1_defence_success_f_attacks = ("{} ± {}".format(avg_l1_distance_failed_attacks_def, std_l1_distance_failed_attacks_def))
    
    resultsDF = resultsDF.append({'model': model_name, 
                                    'benign_confidence': benign_conf, 
                                    'benign_defence_success_rate': benign_defence_success_rate, 
                                    'benign_noise': benign_noise, 
                                    'attack_success_count': attack_success_count_text, 
                                    'noise_success_attacks': noise_success_attacks, 
                                    'attack_failure_count': attack_failure_count_text, 
                                    'noise_failed_attacks': noise_failed_attacks, 
                                    'defence_success_s_attacks': defence_success_s_attacks, 
                                    'noise_s_defence_success_attacks': noise_s_defence_success_attacks, 
                                    'noise_f_defence_success_attacks': noise_f_defence_success_attacks, 
                                    'defence_success_f_attacks': defence_success_f_attacks,  
                                    'noise_defence_success_f_attacks': noise_defence_success_f_attacks, 
                                    'defense_time': defense_time, 
                                    'l1_s_defence_success_attacks': l1_s_defence_success_attacks, 
                                    'l1_f_defence_success_attacks': l1_f_defence_success_attacks, 
                                    'l1_defence_success_f_attacks': l1_defence_success_f_attacks, 
                                    }, ignore_index=True)
    return resultsDF

In [22]:
model_resnet18 = models.resnet18(pretrained=True) 
model_resnet34 = models.resnet34(pretrained=True) 
model_resnet50 = models.resnet50(pretrained=True) 
model_resnet101 = models.resnet101(pretrained=True) 
model_resnet152 = models.resnet152(pretrained=True) 

model_vgg11 = models.vgg11(pretrained=True)
model_vgg13 = models.vgg13(pretrained=True)
model_vgg16 = models.vgg16(pretrained=True)
model_vgg19 = models.vgg19(pretrained=True)

model_densenet121 = models.densenet121(pretrained=True)   
model_densenet161 = models.densenet161(pretrained=True) 
model_densenet169 = models.densenet169(pretrained=True) 
model_densenet201 = models.densenet201(pretrained=True) 

In [23]:
target_models_list = [model_resnet18, model_resnet34, model_resnet50, model_resnet101, model_resnet152, 
                      model_vgg11, model_vgg13, model_vgg16, model_vgg19, 
                      model_densenet121, model_densenet161, model_densenet169, model_densenet201]


target_models_names = ["resnet18", "resnet34", "resnet50", "resnet101", "resnet152",
                       "vgg11", "vgg13", "vgg16", "vgg19",
                       "densenet121", "densenet161", "densenet169", "densenet201"] 

In [ ]:
columns_list = ['model', 'benign_confidence', 'benign_defence_success_rate', 'benign_noise', 
               'attack_success_count', 'noise_success_attacks', 'attack_failure_count', 
               'noise_failed_attacks', 'defence_success_s_attacks', 'noise_s_defence_success_attacks', 
               'noise_f_defence_success_attacks', 'defence_success_f_attacks', 'noise_defence_success_f_attacks', 
               'defense_time', 'l1_s_defence_success_attacks', 'l1_f_defence_success_attacks', 
               'l1_defence_success_f_attacks'] 

results =  pd.DataFrame(columns = columns_list)

for i in range(0, len(target_models_list)): 
    criterion = nn.CrossEntropyLoss()

    # Create the ART classifier
    classifier = PyTorchClassifier(
        model=target_models_list[i],
        loss=criterion,
        input_shape=(3, 224, 224),
        nb_classes=1000,
        device_type='cuda:0'
    )
    
    ## defenses 
    ## bits_squeezing = BitSqueezing(bit_depth=4) 
    median_filter = MedianSmoothing2D(kernel_size=2)
    
    boundary_attack = BoundaryAttack(estimator = classifier, epsilon=0.01, max_iter=1000, targeted=False, verbose=False)
#     pgd_attack = ProjectedGradientDescent(classifier, max_iter=20, eps_step=1, eps=0.01, verbose=False) 
#     fgsm_attack = FastGradientMethod(estimator = classifier, eps=0.01)
#     newton_attack = NewtonFool(classifier=classifier, max_iter=5, verbose=False) 
#     attack_simba = SimBA(classifier=classifier, epsilon = 0.05, max_iter=5000)
#     cw_attack = CarliniL2Method(classifier=classifier, max_iter=2, learning_rate=0.05, verbose=False)
#     hopskipjump_attack = HopSkipJump(classifier=classifier, max_iter=20, verbose=False) 
    result_table = experiment1_method(classifier, boundary_attack, median_filter, loaded_image_paths_list)  
    
    result_table.to_csv("exp4_results/{}_boundary_attack_median_filter.csv".format(target_models_names[i]), index=False)
    results = summarize_result(result_table, 200, results, target_models_names[i])
    print("Target model: {} - Done".format(target_models_names[i]))

Target model: resnet18 - Done
Target model: resnet34 - Done


In [52]:
results.to_csv("RESULTS_EXPERIMENT_4_TABLE/imagenet_experiment1_boundary_attack_median_filter.csv")  

#### For error case 

In [24]:
columns_list = ['model', 'benign_confidence', 'benign_defence_success_rate', 'benign_noise', 
               'attack_success_count', 'noise_success_attacks', 'attack_failure_count', 
               'noise_failed_attacks', 'defence_success_s_attacks', 'noise_s_defence_success_attacks', 
               'noise_f_defence_success_attacks', 'defence_success_f_attacks', 'noise_defence_success_f_attacks', 
               'defense_time', 'l1_s_defence_success_attacks', 'l1_f_defence_success_attacks', 
               'l1_defence_success_f_attacks'] 

results =  pd.DataFrame(columns = columns_list)

for i in range(0, len(target_models_list)): 
    result_table = pd.read_csv("exp4_results/{}_fgsm_bits_squeezing.csv".format(target_models_names[i]))
    results = summarize_result(result_table, 1000, results, target_models_names[i])
    
results 

,model,benign_confidence,benign_defence_success_rate,benign_noise,attack_success_count,noise_success_attacks,attack_failure_count,noise_failed_attacks,defence_success_s_attacks,noise_s_defence_success_attacks,noise_f_defence_success_attacks,defence_success_f_attacks,noise_defence_success_f_attacks,defense_time,l1_s_defence_success_attacks,l1_f_defence_success_attacks,l1_defence_success_f_attacks
0,resnet18,0.945 ± 0.076,999/1000 (0.934±0.097),0.041 ± 0.0147,947/1000 (0.48±0.298),0.014 ± 0.004,53/1000 (0.547±0.278),0.015 ± 0.005,27/947 (0.38±0.167),0.061 ± 0.022,0.06 ± 0.018,53/53 (0.644±0.263),0.06 ± 0.023,0.0004 ± 0.0003,0.046 ± 0.012,0.055 ± 0.006,0.053 ± 0.009
1,resnet34,0.97 ± 0.055,998/1000 (0.966±0.063),0.041 ± 0.0147,900/1000 (0.551±0.29),0.014 ± 0.004,100/1000 (0.574±0.286),0.015 ± 0.005,47/900 (0.398±0.185),0.065 ± 0.019,0.06 ± 0.018,99/100 (0.657±0.277),0.059 ± 0.023,0.0005 ± 0.0026,0.05 ± 0.011,0.055 ± 0.005,0.052 ± 0.01
2,resnet50,0.973 ± 0.053,1000/1000 (0.968±0.073),0.041 ± 0.0147,811/1000 (0.539±0.291),0.014 ± 0.004,189/1000 (0.569±0.284),0.015 ± 0.005,55/811 (0.415±0.197),0.067 ± 0.021,0.059 ± 0.018,188/189 (0.648±0.276),0.062 ± 0.023,0.0005 ± 0.0031,0.051 ± 0.01,0.055 ± 0.005,0.053 ± 0.009
3,resnet101,0.981 ± 0.045,999/1000 (0.979±0.054),0.041 ± 0.0147,764/1000 (0.566±0.283),0.014 ± 0.004,236/1000 (0.632±0.259),0.015 ± 0.005,55/764 (0.422±0.2),0.067 ± 0.02,0.059 ± 0.018,232/236 (0.718±0.236),0.059 ± 0.021,0.0005 ± 0.0014,0.052 ± 0.009,0.055 ± 0.005,0.053 ± 0.009
4,resnet152,0.984 ± 0.04,1000/1000 (0.983±0.044),0.041 ± 0.0147,715/1000 (0.576±0.278),0.014 ± 0.004,285/1000 (0.627±0.268),0.015 ± 0.005,68/715 (0.447±0.161),0.067 ± 0.017,0.059 ± 0.017,281/285 (0.71±0.247),0.059 ± 0.023,0.0006 ± 0.0019,0.053 ± 0.009,0.055 ± 0.004,0.053 ± 0.009
5,vgg11,0.905 ± 0.095,997/1000 (0.882±0.128),0.041 ± 0.0147,944/1000 (0.426±0.273),0.014 ± 0.004,56/1000 (0.429±0.267),0.015 ± 0.005,20/944 (0.341±0.196),0.064 ± 0.026,0.06 ± 0.018,54/56 (0.495±0.267),0.058 ± 0.025,0.0004 ± 0.0002,0.05 ± 0.014,0.054 ± 0.006,0.052 ± 0.011
6,vgg13,0.933 ± 0.084,998/1000 (0.926±0.099),0.041 ± 0.0147,942/1000 (0.454±0.28),0.014 ± 0.004,58/1000 (0.478±0.225),0.015 ± 0.006,30/942 (0.356±0.179),0.066 ± 0.023,0.06 ± 0.018,56/58 (0.579±0.232),0.056 ± 0.03,0.0004 ± 0.0004,0.051 ± 0.011,0.054 ± 0.006,0.051 ± 0.012
7,vgg16,0.959 ± 0.061,1000/1000 (0.949±0.084),0.041 ± 0.0147,915/1000 (0.469±0.28),0.014 ± 0.004,85/1000 (0.52±0.254),0.015 ± 0.006,38/915 (0.449±0.186),0.067 ± 0.021,0.06 ± 0.017,82/85 (0.644±0.243),0.058 ± 0.028,0.0004 ± 0.0004,0.051 ± 0.01,0.055 ± 0.005,0.051 ± 0.011
8,vgg19,0.96 ± 0.066,996/1000 (0.952±0.08),0.041 ± 0.0147,887/1000 (0.464±0.283),0.014 ± 0.004,113/1000 (0.544±0.236),0.015 ± 0.006,46/887 (0.388±0.179),0.068 ± 0.022,0.06 ± 0.017,110/113 (0.633±0.244),0.058 ± 0.025,0.0004 ± 0.0004,0.051 ± 0.011,0.055 ± 0.005,0.052 ± 0.009
9,densenet121,0.971 ± 0.065,1000/1000 (0.968±0.069),0.041 ± 0.0147,808/1000 (0.545±0.294),0.014 ± 0.004,192/1000 (0.554±0.27),0.015 ± 0.005,64/808 (0.398±0.163),0.066 ± 0.023,0.06 ± 0.018,186/192 (0.672±0.257),0.058 ± 0.021,0.0004 ± 0.0006,0.052 ± 0.009,0.055 ± 0.005,0.052 ± 0.009


In [25]:
# results.to_csv("RESULTS_EXPERIMENT_4_TABLE/imagenet_experiment1_fgsm_bits_squeezing.csv") 